<a href="https://colab.research.google.com/github/zuzannazak/Tangles/blob/main/Tangle_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tangles theoretical intro

##Definitions

Traditional clustering identifies groups of objects that share
certain qualities. Tangles do the converse: they identify
groups of qualities that often occur together. They can
thereby identify and discover types: of behaviour, views,
abilities, dispositions.



*   (potential) features s of (elements v of) V (aka separations)
   *    The elements of S are called potential features of (the elements of) V, those of S~ are their features. An element of V has a feature A if it lies in A.
*   specifications →s , ←s of such s
   *   A subset of S~ is a specification of S if for every s ∈ S it contains either →s or ←s but not both. The elements of S are called potential features of (the elements of) V, those of S~ are their features. An element of V has a feature A if it lies in A.
   *   types (or typical specifications of S) 
      *   A subset of S~ is a specification of S if for every s ∈ S it contains either →s or ←s but not both.
   *   consistency of specifications
     *   A subset of S~ is inconsistent if it contains three features →r, →s, →t such that no element of V has them al
*   tangles
   *   Every Fn-tangle of S contains all the features ←s for which →s has fewer than n elements, because it cannot contain {→s } ∈ Fn as a subset.
*   star
   *   A set of features is a star if their inverses are disjoint subsets of V. 
*   universe of features
   *   We shall be particularly interested in sets S such that S~ contains, for any given →r ,→s ∈ S~, also the features →r ∨ →s and →r ∧ →s . If S has this property, then S~ will, in fact, contain all Boolean expressions of features it contains already - combinations of features built from others by using the symbols ∨ and ∧ and taking inverses. We then call S~ a universe of features.
*   submodularity
   *   S and S~ are submodular if for any two features →r ,→s ∈ S~ at least one of →r ∨ →s and →r ∧ →s is also in S~.



#Tangles example - Polish administrative regions and numeric variables

Data about Polish administrative regions (16) regarding population health.

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd
import numpy as np

In [2]:
#df = pd.read_excel("/content/drive/MyDrive/STUDIA/Doktorskie/Tangle/test-data.xlsx")
df = pd.read_excel("https://github.com/zuzannazak/Tangles/blob/main/wojewodztwa_test_data.xlsx?raw=true")
df.head()

,Nazwa wojewodztwa,Oczekiwane trwanie życia w zdrowiu - mężczyźni,Oczekiwane trwanie życia w zdrowiu - kobiety,Lekarze posiadający prawo wykonywania zawodu medycznego na 10 tys. ludności,Powierzchnia gruntów leśnych (ha)
0,DOLNOŚLĄSKIE,59.3,63.1,43.2,28652.24
1,KUJAWSKO-POMORSKIE,59.8,63.5,32.3,53390.76
2,LUBELSKIE,59.4,62.9,42.0,240792.69
3,LUBUSKIE,60.6,64.2,25.3,15003.21
4,ŁÓDZKIE,58.7,62.7,48.3,139044.29


#Tangles example - fake yes/no questionnaire

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd
import numpy as np

In [4]:
#df = pd.read_excel("/content/drive/MyDrive/STUDIA/Doktorskie/Tangle/Tangles_example_fake_data.xlsx")
df = pd.read_excel("https://github.com/zuzannazak/Tangles/blob/main/Tangles_example_fake_data.xlsx?raw=true")
df.head()

,Person,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10
0,A,Yes,Yes,No,No,No,No,Yes,No,No,Yes
1,B,Yes,Yes,No,Yes,Yes,No,No,Yes,Yes,Yes
2,C,Yes,Yes,No,No,Yes,No,Yes,Yes,No,Yes
3,D,Yes,Yes,No,Yes,No,No,No,No,Yes,Yes
4,E,Yes,Yes,No,No,Yes,No,No,Yes,Yes,Yes


In [10]:
#calculating cost function for cuts - the mean over the similarities over all possible pairs of separated persons:

list_of_cost_func = {}
for columnName in df.iteritems():
  qA = df.loc[df[str(columnName[0])] == "Yes"]
  qB = df.loc[df[str(columnName[0])] == "No"]

  qA = qA.reset_index(drop=True)
  qB = qB.reset_index(drop=True)


  sum_of_similarities = 0

  for i, row in qA.iterrows():
      for l, row in qB.iterrows():
        for k in range(10):
          if qB.iat[l,k] == qA.iat[i,k]: 
            sum_of_similarities = sum_of_similarities +1
            #print(str(j) + ": rząd " + str(l) + ", pytanie " + str(k) + ": " + q1B.iat[l,k] + " i "+"rząd " + str(i) +", pytanie " + str(k) + ": " + q1A.iat[i,k])
            #print(sum_of_similarities)

  number_of_pairs_between_sets = qA.shape[0]*qB.shape[0]
  if number_of_pairs_between_sets == 0:
    number_of_pairs_between_sets = 0.0000001
  normalized_func_cost = sum_of_similarities/number_of_pairs_between_sets
  list_of_cost_func[str(columnName[0])] = normalized_func_cost

  sum_of_similarities = 0


list_of_cost_func
#why 0 when all of them are different, or all of them are the same?

{'Person': 0.0,
 'Question 1': 4.641666666666667,
 'Question 10': 5.225,
 'Question 2': 0.0,
 'Question 3': 4.404761904761905,
 'Question 4': 4.823076923076923,
 'Question 5': 4.76984126984127,
 'Question 6': 4.531746031746032,
 'Question 7': 4.785714285714286,
 'Question 8': 4.761904761904762,
 'Question 9': 4.7272727272727275}

In [17]:
desired_value = 0
for key, value in list_of_cost_func.items():
  if value == desired_value:
    del list_of_cost_func[key]
    break

list_of_cost_func


{'Question 1': 4.641666666666667,
 'Question 10': 5.225,
 'Question 3': 4.404761904761905,
 'Question 4': 4.823076923076923,
 'Question 5': 4.76984126984127,
 'Question 6': 4.531746031746032,
 'Question 7': 4.785714285714286,
 'Question 8': 4.761904761904762,
 'Question 9': 4.7272727272727275}

##Naive table search algorythm
**Input:** The separation system S, enumerated as s1, . . . , sn; oracle access to F.

**Output:** For every i ≤ n a list Ti of all F-tangles of {s1, . . . , si}.